In [1]:
import pandas as pd
import os
from multiprocessing import  Pool
import multiprocessing
import datetime


In [2]:
df = pd.read_parquet("data/mstatsdf.parquet")
df.describe()

,azure_brazilsouth,gcp_southamerica-east1,aws_sa-east-1,aws_ca-central-1,gcp_northamerica-northeast1,aws_us-east-1,azure_eastus,gcp_us-east4,aws_us-west-1,aws_us-west-2,...,gcp_europe-west3,aws_ap-south-1,gcp_asia-south1,azure_centralindia,aws_ap-northeast-1,gcp_asia-northeast1,azure_japaneast,aws_ap-southeast-2,azure_australiaeast,gcp_australia-southeast1
count,3054.000000,3054.000000,3054.000000,3054.000000,3054.0,3054.000000,3054.000000,3054.0,3054.000000,3054.000000,...,3054.0,3054.000000,3054.0,3054.000000,3054.000000,3054.000000,3054.000000,3054.000000,3054.000000,3054.0
mean,0.993124,0.999673,0.997708,0.998690,1.0,0.995416,0.992141,1.0,0.997380,0.995088,...,1.0,0.997708,1.0,0.998035,0.998035,0.999673,0.998035,0.998363,0.994434,1.0
std,0.082651,0.018095,0.054246,0.036173,0.0,0.067562,0.111288,0.0,0.051122,0.069922,...,0.0,0.047829,0.0,0.044288,0.044288,0.018095,0.057198,0.040436,0.074413,0.0
min,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,0.000000,1.0,0.000000,0.000000,...,1.0,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0
25%,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.0,1.000000,1.000000,...,1.0,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0
50%,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.0,1.000000,1.000000,...,1.0,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0
75%,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.0,1.000000,1.000000,...,1.0,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0
max,1.000000,1.000000,2.000000,1.000000,1.0,1.000000,2.000000,1.0,1.000000,1.000000,...,1.0,1.000000,1.0,1.000000,1.000000,1.000000,2.000000,1.000000,1.000000,1.0


In [34]:
# Count Per Region How Many Measurements Are Affected:
df_cnt_zeroes = df
df_cnt_zeroes

totals = {
    "AWS": {"OK": 0, "MISS": 0, "ADD": 0},
    "AZURE": {"OK": 0, "MISS": 0, "ADD": 0},
    "GCP": {"OK": 0, "MISS": 0, "ADD": 0},
}

for pr in df_cnt_zeroes.columns:
    p = pr.split('_')[0].upper()
    r = pr.split('_')[1]
    ok_measurement = df_cnt_zeroes[df_cnt_zeroes[pr] == 1][pr].count()
    missing_measurement = df_cnt_zeroes[df_cnt_zeroes[pr] == 0][pr].count()
    more_measurements = df_cnt_zeroes[df_cnt_zeroes[pr] > 1][pr].count()
    totals[p]['OK']   += ok_measurement
    totals[p]['MISS'] += missing_measurement
    totals[p]['ADD']  += more_measurements
    print(p, ' & ', r, ' & ' , ok_measurement, ' & ', missing_measurement, ' & ', more_measurements, ' \\\\')

pr = 'azure_brazilsouth'
totals

AZURE  &  brazilsouth  &  3033  &  21  &  0  \\
GCP  &  southamerica-east1  &  3053  &  1  &  0  \\
AWS  &  sa-east-1  &  3045  &  8  &  1  \\
AWS  &  ca-central-1  &  3050  &  4  &  0  \\
GCP  &  northamerica-northeast1  &  3054  &  0  &  0  \\
AWS  &  us-east-1  &  3040  &  14  &  0  \\
AZURE  &  eastus  &  3016  &  31  &  7  \\
GCP  &  us-east4  &  3054  &  0  &  0  \\
AWS  &  us-west-1  &  3046  &  8  &  0  \\
AWS  &  us-west-2  &  3039  &  15  &  0  \\
AZURE  &  westus  &  2996  &  58  &  0  \\
AZURE  &  westus2  &  3048  &  6  &  0  \\
GCP  &  us-west2  &  3053  &  1  &  0  \\
GCP  &  us-west4  &  3053  &  1  &  0  \\
AWS  &  eu-west-2  &  3046  &  8  &  0  \\
AZURE  &  uksouth  &  3042  &  8  &  4  \\
GCP  &  europe-west2  &  3054  &  0  &  0  \\
AWS  &  eu-central-1  &  3048  &  6  &  0  \\
AZURE  &  germanywestcentral  &  3049  &  4  &  1  \\
GCP  &  europe-west3  &  3054  &  0  &  0  \\
AWS  &  ap-south-1  &  3047  &  7  &  0  \\
GCP  &  asia-south1  &  3054  &  0  &  0  \\
A

{'AWS': {'OK': 30458, 'MISS': 81, 'ADD': 1},
 'AZURE': {'OK': 27313, 'MISS': 159, 'ADD': 14},
 'GCP': {'OK': 30536, 'MISS': 4, 'ADD': 0}}